

* **1. Data preparation**
    * 1.1 Load data
    * 1.2 Check for null and missing values
    * 1.3 Normalization
    * 1.4 Reshape
    * 1.5 Label encoding
    * 1.6 Split training and valdiation set
* **2. CNN**
    * 2.1 Define the model
    * 2.2 Set the optimizer and annealer
    * 2.3 Data augmentation
* **3. Evaluate the model**
    * 3.1 Fitting the CNN to the Image

# ***Intro***
# 1.1 Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

X = train.drop('label', axis=1)
y = train['label']

# 1.2 Check for null and missing values

In [ ]:
train.isnull().sum().any()
test.isnull().sum().any()

In [ ]:
sns.countplot(x = y)

# 1.4 Normalization

In [ ]:
X = X / 255.0
test = test / 255.0

 # 1.3 Reshape

In [ ]:
X.shape

In [ ]:
X = X.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

The train and test dataframe has correspond to 1D which we try to reshape all photos to height = 28px , width = 28px, channel=3 (RGB).

# 1.4 Label encoding

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
y = to_categorical(y, num_classes = 10)

# 1.5 Split training and valdiation set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# **2. CNN**
# 2.1 Define the model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [ ]:
model = Sequential()

model.add(Conv2D(32, (5, 5), padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(32, (5,5), padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding = 'Same', activation ='relu'))
model.add(Conv2D(64, (3,3), padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

# ***3. Evaluate the model***
## 3.1 Fitting the CNN to the Image

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=86),
                              epochs = 20, validation_data = (X_test, y_test),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // 86
                              , callbacks=[learning_rate_reduction])

In [ ]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)